## Analyzing the impact of weather on bike trip duration using OLS regression method



In [1]:
# All Libraries needed for Regression imported
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', 999)
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# Reading DFs as CSV needed for regression
bike_journey_data=pd.read_csv('bike_journey_data_27_mar_2020.csv')
weather_data=pd.read_csv('weather_data_27_mar_2020.csv')

In [18]:
#Collapsing bike journeys by hour and day (24 hours x 365 days) 
bike_journeys=bike_journey_data.loc[bike_journey_data['Duration in minutes'] < 300]
bike_journeys['c'] = 1
daily_bike_journeys = bike_journeys.loc[:, ('c', 'id', 'id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
daily_bike_journeys.reset_index(inplace=True)
daily_bike_journeys.rename(columns={'c': 'Bike trips', 'id': 'id', 'Hours': 'Hours'}, inplace=True)
#print (daily_bike_journeys.shape)

#Merging bike journeys with the weather data by hour and day (24 hours x 365 days) 
reg_data= daily_bike_journeys.merge(right=weather_data,
                             left_on = 'id_hours',
                             right_on = 'id_Hours')

#Generates one of the dependent variables "Avg trip duration"/January 2019 is used as a reference group
reg_data['Avg trip duration']=reg_data['Duration in minutes']/reg_data['Bike trips']

#Generates dummy variables for months of the year to control for seasonality in the regression 
df_month=pd.get_dummies(reg_data.Month, drop_first=True)
df_month.columns=['February 2019','March 2019', 'April 2019','May 2019','June 2019','July 2019',
                  'August 2019','September 2019','October 2019','November 2019','December 2019']
reg_data = reg_data.join(df_month)

#Creates dummy variables for temperature by cutting it into 5°C bins/Temp - 15(°C) is used as a reference group
reg_data["Temp - 0s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x <=0 else 0)
reg_data["Temp - 5s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>0 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=5 else 0)
reg_data["Temp - 10s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>5 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=10 else 0)
#reg_data["Temp - 15(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>10 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=15 else 0)
reg_data["Temp - 20s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>15 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=20 else 0)
reg_data["Temp - 25s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>20 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=25 else 0)
reg_data["Temp - 30s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>25 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=30 else 0)#reg_data["temp_35"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=35 else 0)
reg_data["Temp - 35s(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=38 else 0)

#Creates dummy variables for humidity, rain, wind speed, visibility, weekend, holidays, peak travel hours
reg_data["Relative Humidity"] = reg_data['humidity'].apply(lambda x: 1 if x >=0.6497 else 0)
reg_data["Rain"] = reg_data['precipIntensity'].apply(lambda x: 1 if x >0.0697 else 0)
reg_data["Wind Speed (MPH)"] = reg_data['windSpeed'].apply(lambda x: 1 if x >8.1400 else 0)
reg_data["visibility"] = reg_data['visibility'].apply(lambda x: 1 if x >15.4914 else 0)
reg_data["Weekend"] = reg_data['Week Day'].apply(lambda x: 1 if x >4 else 0)
reg_data["Peak travel hours"] = reg_data['Hours'].apply(lambda x: 1 if x >=7 and x<10 else (1 if x >= 16 and x < 20 else 0))
reg_data=reg_data.set_index('id')
holidays = ['2019-1-1', '2019-4-19', '2019-4-22', '2019-5-6', '2019-5-27', '2019-8-26', '2019-12-25', '2019-12-26']
reg_data['Holidays'] = reg_data.index.get_level_values(0).isin(holidays).astype(int)

/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Generates descriptive statistics for key dependent and independent variables in the regression

# Trips per hour
Trips_hour = reg_data['Bike trips'].describe()
Trips_hour

# Trip duration per hour
Avg_trip_duration = reg_data['w'].describe()
Avg_trip_duration

# Teperature
Teperature = reg_data['temperature'].describe()
Teperature

# Humidity
Humidity = reg_data['Relative Humidity'].describe()
Humidity

# Rain
Rain = reg_data['Rain'].describe()
Rain

# Wind Speed
Wind_Speed_MPH= reg_data['Wind Speed (MPH)'].describe()
Wind_Speed_MPH

# Visibility
Visibility= reg_data['visibility'].describe()
Visibility

# Peak Travel Hours
Peak_travel_hours = reg_data['Peak travel hours'].describe()
Peak_travel_hours

#Weekends
Weekend = reg_data['Weekend'].describe()
Weekend

#Holidays
Holidays = reg_data['Holidays'].describe()
Holidays

In [19]:
#Assigning the dependent and independent variables for the OLS regression and running the model

y1 = 'Avg trip duration'
y2 = 'Bike trips'

x_0=['temperature']

x_1 = ["temperature",'February 2019','March 2019','April 2019','May 2019','June 2019','July 2019',
       'August 2019','September 2019','October 2019','November 2019','December 2019', "Peak travel hours",
       "Holidays", "Weekend", 'visibility']

x_2 = ["Temp - 0s(°C)", "Temp - 5s(°C)", "Temp - 10s(°C)", "Temp - 15s(°C)", "Temp - 20s(°C)", "Temp - 25s(°C)",
       "Temp - 30s(°C)", "Temp - 35s(°C)"]

x_3 = ["Temp - 0s(°C)", "Temp - 5s(°C)", "Temp - 10s(°C)", "Temp - 20s(°C)", "Temp - 25s(°C)",
       "Temp - 30s(°C)", "Temp - 35s(°C)", "Relative Humidity", "Rain", 'Wind Speed (MPH)','February 2019','March 2019', 
       'April 2019','May 2019','June 2019','July 2019','August 2019','September 2019','October 2019',
       'November 2019','December 2019', "Peak travel hours", "Holidays", "Weekend",'visibility']

model_0 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_0])).fit()
model_1 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_1])).fit()
model_2 = sm.OLS(reg_data[y2], sm.add_constant(reg_data[x_3])).fit()
model_3 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_3])).fit()

/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [20]:
# Outputing the regression results
dfoutput = summary_col([model_3, model_2],stars=True, 
                       regressor_order=['temperature',"Temp - 0s(°C)", "Temp - 5s(°C)", "Temp - 10s(°C)", "Temp - 20s(°C)", "Temp - 25s(°C)",
                                        "Temp - 30s(°C)", "Temp - 35s(°C)", "Relative Humidity", "Rain", 'Wind Speed (MPH)','visibility', "Weekend",'Holidays',
                                        "Peak travel hours",'February 2019','March 2019','April 2019','May 2019','June 2019','July 2019',
                                        'August 2019','September 2019','October 2019','November 2019','December 2019'])

dfoutput.add_title('Table 1: OLS Model for Weather Impacts on Bicycling Behavior')

print(dfoutput)

Table 1: OLS Model for Weather Impacts on Bicycling Behavior
                  Avg trip duration  Bike trips 
------------------------------------------------
Temp - 0s(°C)     -2.3462***        -565.4141***
                  (0.4147)          (94.7159)   
Temp - 5s(°C)     -1.8217***        -414.8467***
                  (0.1641)          (37.4881)   
Temp - 10s(°C)    -1.2821***        -280.7228***
                  (0.1099)          (25.0912)   
Temp - 20s(°C)    1.6149***         393.5553*** 
                  (0.1197)          (27.3322)   
Temp - 25s(°C)    4.2233***         869.2323*** 
                  (0.1763)          (40.2584)   
Temp - 30s(°C)    4.4926***         1090.1384***
                  (0.3490)          (79.7037)   
Temp - 35s(°C)    5.6232***         1173.2252***
                  (0.4706)          (107.4911)  
Relative Humidity -1.5351***        -558.9768***
                  (0.1012)          (23.1194)   
Rain              -1.3278***        -119.9598***
        